I really wanna do this without anaconda. It always messing with my system somehow.

## Install *pytorch* (cuda enabled) with *pip*

- go to installation folder\
```cd c:/dev```

- select cuda and pytorch version with this helpfool tool:\
https://pytorch.org/get-started/locally/

- install pytorch with cuda enabled (copy and paste the code from the above tool)\
```pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116```

- fix longpaths on windows if installation fails
- run powershell with admin\
```New-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\FileSystem" `
-Name "LongPathsEnabled" -Value 1 -PropertyType DWORD -Force```

- check environment\
```python -m torch.utils.conda_env```

In [ ]:
"""Check pytorch version"""

import torch
print("VERSION")
print("pytorch:     ", torch.__version__)
print("cuda enabled:", torch.cuda.is_available())
import detectron2
print("detectron:   ", detectron2.__version__)

In [ ]:
'''Import standard libraries'''
from typing import List, Union, Tuple
import os
import sys
import numpy as np
import json, cv2, random, requests
import matplotlib.pyplot as plt

In [ ]:
'''Import detectron and certain utilities'''

import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

#from google.colab.patches import cv2_imshow

## Detectron
preconfigured models: https://github.com/facebookresearch/detectron2/blob/main/MODEL_ZOO.md

In [ ]:
'''Create detectron preconfigured model'''

cfg = get_cfg()
baselines = {
    "R50-FPN": "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml",
    "X101-FPN": "COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml",
    "R101-FPN-panoptic": "https://github.com/facebookresearch/detectron2/blob/main/configs/COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
}
cfg.merge_from_file(model_zoo.get_config_file(baselines["R101-FPN-panoptic"]))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(baselines["R101-FPN-panoptic"])

predictor = DefaultPredictor(cfg)

In [ ]:
'''Inference image'''
# Image A
#image_in = cv2.imread(str(filenames[0]))

# Image B
from PIL import Image
import requests
url = "https://images.contentstack.io/v3/assets/blt00454ccee8f8fe6b/bltf8a72260189be996/60ab28704adeb65c03d761d4/UK_Budapest_Hungary_Header.jpg?width=1680&quality=70&auto=webp"
image_in = Image.open(requests.get(url, stream=True).raw).resize((640,480))
image_in = np.array(image_in)

outputs = predictor(image_in)
instances = outputs["instances"].to("cpu")

'''visualize with default visualizer'''
v = Visualizer(image_in[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
image_out = v.draw_instance_predictions(instances).get_image()[:, :, ::-1] # prediction to np_image
plt.axis("off")
plt.imshow(image_out)

## Examine predictions

In [ ]:
instances = outputs["instances"].to("cpu")

print("prediction fields:", ", ".join(instances.get_fields().keys()) )

classes = instances.get("pred_classes")
boxes = instances.get("pred_boxes")
masks = instances.get("pred_masks")

print("classes:", classes)
print("boxes:", boxes)

plt.imshow(masks[0])

In [ ]:
'''Examine catalog'''

MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_colors

import pandas as pd
thing_ids = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id
thing_classes = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes
pd.DataFrame( data=thing_classes, index=thing_ids, columns=["label"] )

# Custom visualization

In [ ]:
from detectron2.utils.visualizer import Visualizer

# Create catalog
catalog = dict(zip(
    MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id, 
    MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes)
    )

# draw masks
v = Visualizer(np.zeros(shape=image_in.shape), catalog.keys(), scale=1.0)
classes = outputs["instances"].pred_classes.to('cpu')
boxes = outputs["instances"].pred_boxes.to('cpu')
masks = outputs["instances"].pred_masks.to('cpu')

for thing_id, box, soft_mask in zip(classes, boxes, masks):
    if thing_id.item()==0: # person
        v.draw_soft_mask(soft_mask, color=(1,0,0,1), alpha=1.0)
    if thing_id.item()==17: # horse
        v.draw_soft_mask(soft_mask, color=(1,0,0,1), alpha=1.0)

# get image
v = v.get_output()
image_out = v.get_image()

# show masks
print(image_in.shape, image_in.dtype)
print(image_out.shape, image_out.dtype)
plt.imshow(image_in//2+image_out//2)



In [ ]:
# OO version

class SemanticSegmentation:
    catalog = dict(zip(
        MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_dataset_id_to_contiguous_id, 
        MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes)
    )
    def __init__(self, image_in):
        self.image_in = image_in

    def eval(self):
        v = Visualizer(np.zeros(shape=self.image_in.shape), catalog.keys(), scale=1.0)
        classes = outputs["instances"].pred_classes.to('cpu')
        boxes = outputs["instances"].pred_boxes.to('cpu')
        masks = outputs["instances"].pred_masks.to('cpu')

        for thing_id, box, soft_mask in zip(classes, boxes, masks):
            if thing_id.item()==0: # person
                v.draw_soft_mask(soft_mask, color=(1,0,0,1), alpha=1.0)
            if thing_id.item()==17: # horse
                v.draw_soft_mask(soft_mask, color=(1,0,0,1), alpha=1.0)

        # get image
        v = v.get_output()
        image_out = v.get_image()
        return image_out

masks = SemanticSegmentation(image_in).eval()
plt.imshow(image_in//2+masks//2)